In [80]:
import pandas as pd
import joblib   
import numpy as np
from sklearn.model_selection import train_test_split
from SimulateDay import scale_data, predict_action, get_stock_data, add_columns, stock_market_simulation
import xgboost as xgb
import joblib
import altair as alt
import yfinance as yf

def get_stock_movement(symbol):
    data = yf.download(symbol, period="1y")
    if len(data) > 0:
        initial_price = data['Close'].iloc[0]
        latest_price = data['Close'].iloc[-1]
        return ((latest_price - initial_price) / initial_price) * 100
    return 0

import plotly.subplots as sp
import plotly.graph_objects as go
def get_final_portfolio_values(df):
    # Group by 'Stock Name' and get the last row for each group
    final_values = df.groupby('Stock Name').apply(lambda x: x.iloc[-1])
    
    # Extract 'Stock Name' and 'Portfolio Value' columns
    result = final_values[['Stock Name', 'Portfolio Value','Shares Held']].reset_index(drop=True)
    
    return result




In [110]:
LGBM_decisions = pd.read_csv('simResults/LGBM_model_decisions.csv')
sim_results = pd.read_csv('simResults/sim_results.csv')
specific_model_decisions = pd.read_csv('simResults/specific_model_decisions.csv').sort_values(by=['Stock Name', 'Day'])
general_model_decisions = pd.read_csv('simResults/general_model_decisions.csv').sort_values(by=['Stock Name', 'Day'])

In [104]:
specific_model_decisions.head(10)

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
0,A,0,Hold,144.800003,10000.000000,0.0,10000.0,2024-10-09
1,A,1,Sell,142.740005,10000.000000,0.0,10000.0,2024-10-10
2,A,2,Sell,143.820007,10000.000000,0.0,10000.0,2024-10-11
3,A,3,Sell,144.250000,10000.000000,0.0,10000.0,2024-10-14
4,A,4,Sell,144.580002,10000.000000,0.0,10000.0,2024-10-15
5,A,5,Sell,138.830002,10000.000000,0.0,10000.0,2024-10-16
6,A,6,Sell,138.245697,10000.000000,0.0,10000.0,2024-10-17
7,A,7,Sell,138.774994,10000.000000,0.0,10000.0,2024-10-18
4214,A,8,Sell,136.869995,10000.000000,0.0,10000.0,2024-10-21
8,AAPL,0,Buy,229.539993,9770.460007,1.0,10000.0,2024-10-09


In [105]:
LGBM_decisions[LGBM_decisions['Action'] == 'Buy']

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
4,MMM,4,Buy,135.679993,9864.320007,1,10000.000000,2024-10-17
10,AOS,2,Buy,80.949997,9919.050003,1,10000.000000,2024-10-15
11,AOS,3,Buy,80.230003,9838.820000,2,9999.280006,2024-10-16
20,ABT,4,Buy,117.889999,9882.110001,1,10000.000000,2024-10-17
21,ABT,5,Buy,117.889999,9764.220001,2,10000.000000,2024-10-17
...,...,...,...,...,...,...,...,...
4006,ZBRA,6,Buy,373.839996,8871.100006,3,9992.619995,2024-10-17
4007,ZBRA,7,Buy,375.450012,8495.649994,4,9997.450043,2024-10-18
4014,ZBH,6,Buy,104.949997,9895.050003,1,10000.000000,2024-10-17
4020,ZTS,4,Buy,192.389999,9807.610001,1,10000.000000,2024-10-15


In [106]:
specific_model_decisions[specific_model_decisions['Action']  == 'Buy']

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
8,AAPL,0,Buy,229.539993,9770.460007,1.0,10000.000000,2024-10-09
9,AAPL,1,Buy,229.039993,9541.420013,2.0,9999.500000,2024-10-10
37,ACGL,1,Buy,113.900002,9886.099998,1.0,10000.000000,2024-10-10
44,ACN,0,Buy,365.079987,9634.920013,1.0,10000.000000,2024-10-09
48,ACN,4,Buy,368.660004,9266.260010,2.0,10003.580017,2024-10-15
...,...,...,...,...,...,...,...,...
3961,ZBH,4,Buy,104.599998,9895.400002,1.0,10000.000000,2024-10-15
3962,ZBH,5,Buy,105.209999,9894.790001,1.0,10000.000000,2024-10-16
3963,ZBH,6,Buy,105.029999,9894.970001,1.0,10000.000000,2024-10-17
3964,ZBH,7,Buy,106.535004,9893.464996,1.0,10000.000000,2024-10-18


In [107]:
specific_model_decisions[(specific_model_decisions['Portfolio Value'] > 10000) & (specific_model_decisions['Day'] == 7)]

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date
15,AAPL,7,Sell,235.244995,10005.705002,0.0,10005.705002,2024-10-18
51,ACN,7,Buy,375.660004,9259.260010,2.0,10010.580017,2024-10-18
107,AEP,7,Buy,100.625000,9801.654999,2.0,10002.904999,2024-10-18
155,AKAM,7,Buy,106.485001,9790.404999,2.0,10003.375000,2024-10-18
235,AMP,7,Buy,522.979980,8983.780029,2.0,10029.739990,2024-10-18
...,...,...,...,...,...,...,...,...
3484,TFX,7,Buy,238.520004,9527.069992,2.0,10004.110001,2024-10-18
3492,TGT,7,Buy,155.550003,9690.119995,2.0,10001.220001,2024-10-18
3508,TMO,7,Hold,601.643982,9400.750000,1.0,10002.393982,2024-10-18
3612,TXT,7,Buy,89.915001,9823.745003,2.0,10003.575005,2024-10-18


In [108]:
specific_model_decisions.describe()

,Day,Stock Price,Cash,Shares Held,Portfolio Value
count,4483.000000,4483.000000,4483.000000,4483.000000,4483.000000
mean,4.016061,223.015008,9893.414336,0.468719,10000.662681
std,2.582457,526.417136,518.035658,0.782826,8.926406
min,0.000000,7.520000,0.000000,0.000000,9857.950195
25%,2.000000,69.587502,9927.645000,0.000000,10000.000000
50%,4.000000,125.912498,10000.000000,0.000000,10000.000000
75%,6.000000,233.110001,10000.000000,1.000000,10000.000000
max,8.000000,9916.605469,10089.529785,4.000000,10289.005859


In [109]:
LGBM_decisions.describe()

,Day,Stock Price,Cash,Shares Held,Portfolio Value
count,4024.000000,4024.000000,4024.000000,4024.000000,4024.000000
mean,3.500000,223.215227,9880.923389,0.593936,9999.695030
std,2.291573,523.721807,359.670803,0.996451,4.691022
min,0.000000,7.520000,226.419922,0.000000,9953.529968
25%,1.750000,69.627499,9893.474998,0.000000,10000.000000
50%,3.500000,126.895000,10000.000000,0.000000,10000.000000
75%,5.250000,233.245003,10000.000000,1.000000,10000.000000
max,7.000000,9924.400391,10000.709999,10.000000,10150.820312


In [111]:
final_portfolio_values = get_final_portfolio_values(sim_results)#.drop(simResults[simResults['Stock Name']=='NFLX'].index))
final_portfolio_values['Profit %'] = (final_portfolio_values['Portfolio Value'] - 10000) / 10000 * 100
alt.Chart(final_portfolio_values).mark_bar().encode(
    x='Stock Name',
    y='Profit %',
    color=alt.condition(
        alt.datum['Profit %'] > 0,
        alt.value('green'),
        alt.value('red')
    )
).properties(
    title='Final Portfolio Value by Stock',
    width=800,
    height=400
).configure_axis(
    labelAngle=45
).display()

alt.Chart(...)

In [112]:
final_portfolio_values.describe() 

,Portfolio Value,Shares Held,Profit %
count,18.000000,18.000000,18.000000
mean,10840.165268,46.533595,8.401653
std,1414.583655,85.426197,14.145837
min,9921.349977,0.000000,-0.786500
25%,10079.935795,11.750000,0.799358
50%,10197.988786,19.208332,1.979888
75%,10541.500292,41.000000,5.415003
max,15113.500288,374.000000,51.135003


In [113]:
sim_results['Profit %'] = (sim_results['Portfolio Value'] - 10000) / 10000 * 100
sim_results.describe()

,Day,Stock Price,Cash,Shares Held,Portfolio Value,Profit %
count,6570.000000,6570.000000,6570.000000,6570.000000,6570.000000,6570.000000
mean,182.000000,161.080648,7409.005335,25.399711,10350.674098,3.506741
std,105.374048,153.169143,3336.277504,49.513217,1056.824854,10.568249
min,0.000000,9.830000,0.000000,-0.884297,8260.710098,-17.392899
25%,91.000000,40.869999,6386.832497,2.000000,9984.420002,-0.155800
50%,182.000000,105.884998,8907.330009,11.000000,10023.335014,0.233350
75%,273.000000,229.029995,9881.982492,29.000000,10207.664939,2.076649
max,364.000000,771.167419,15191.869995,389.000000,17111.548602,71.115486


In [91]:
final_portfolio_values.describe()

,Portfolio Value,Shares Held,Profit %
count,18.000000,18.000000,18.000000
mean,10756.806290,43.034536,7.568063
std,1230.685076,80.641113,12.306851
min,9915.389959,-0.602926,-0.846100
25%,10065.476938,2.327040,0.654769
50%,10264.278403,14.944983,2.642784
75%,10510.354038,43.250000,5.103540
max,14642.950491,342.000000,46.429505


In [92]:
all_stock_movement = []
total_movement = 0
for stock in sim_results['Stock Name'].unique():
    all_stock_movement.append([stock, get_stock_movement(stock)])
    total_movement += get_stock_movement(stock)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [95]:
total_movement/len(sim_results['Stock Name'].unique())


33.24928991723444